**Inputs**
* ImageCollection(s) containing hazard probabilities for 2030 and 2065

**Outputs**
* Images: two-band (alpha, beta), one pair per magnitude

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWi_ol0l8wYgGe9duqU1yUJzvNkxS9D9KyGTnHYplVR_0pk_zWzRUh4

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
HIST_START = 1980
HIST_END = 2019

In [4]:
hazards = ['ds', 'maxdryspell']
hazard_info = {
    'ds': {
        'num_bins': 100,
        'bin_width': 1
    },
    'maxdryspell': {
        'num_bins': 73,
        'bin_width': 5
    }
}

In [6]:
era_mags = {}
for h in hazards:
    era_mags[h] = ee.ImageCollection('users/tedwongwri/dataportal/hist-magnitudes/' + h)

In [7]:
def alpha_beta(mag_low, mag_high):
# alpha and beta parameters for MAP prob formula
# alpha: historical "successes" plus 1
# beta: historical "failures" plus 1
    successes = ee.Image.constant(0).rename('alpha')
    failures = ee.Image.constant(0).rename('beta')
    for hist_year in range(HIST_START, HIST_END + 1):
        # for pixel == magnitude
        histyear = era_mags[hazardname].filterMetadata('year', 'equals', hist_year).first()
        succ = histyear.round().gte(mag_low).multiply(1).multiply(histyear.round().lt(mag_high).multiply(1))
        successes = successes.add(succ)
        failures = failures.add(succ.multiply(-1).add(1))
        # for pixel gte magnitude
        #successes = successes.add(ehe_histyear.gte(magnitude).multiply(ehe_histyear.lt(magnitude + 5)))
        #failures = failures.add(ehe_histyear.lt(magnitude).add(ehe_histyear.gte(magnitude + 5)).min(1))
    alpha = successes.add(1).set('source', 'era5').set('magnitude_low', mag_low)
    beta = failures.add(1).set('source', 'era5').set('magnitude_high', mag_high)
    ic = ee.ImageCollection([alpha, beta])
    twoband_image = ic.toBands().rename(['alpha', 'beta'])
    return twoband_image

In [11]:
for hazardname in hazards:
    for bin_idx in range(hazard_info[hazardname]['num_bins']):
        mag_low = (bin_idx) * hazard_info[hazardname]['bin_width']
        mag_high = (bin_idx + 1) * hazard_info[hazardname]['bin_width']
        img = alpha_beta(mag_low, mag_high).set('magnitude_low', mag_low).set('magnitude_high', mag_high)
        task = ee.batch.Export.image.toAsset(**{
          'image': img,
          'description': 'alphabeta_{0}_{1}-{2}'.format(hazardname, str(mag_low), str(mag_high)),
          'assetId': 'users/tedwongwri/dataportal/alphabeta/{0}/magnitude_{1}-{2}'.format(hazardname, str(mag_low), str(mag_high)),
          'region': WHOLE_GLOBE,
          'crs': 'EPSG:4326',
          'dimensions': '1440x720'
        })
        task.start()